In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score, mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from lightgbm import LGBMRegressor

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.decomposition import PCA



In [ ]:
##Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Change the current working directory
os.chdir('/content/gdrive/My Drive/CDC_openproject')

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content/gdrive/My Drive/CDC_openproject


# load data

In [ ]:
tabular= pd.read_csv("train.csv")
predict_tab=pd.read_csv("test.csv")

train_img = pd.read_csv("train_image_features.csv")
predict_img  = pd.read_csv("test_image_features.csv")


In [ ]:
tabular.drop_duplicates(subset="id", keep="first", inplace=True)
predict_tab.drop_duplicates(subset="id", keep="first", inplace=True)

tabular.drop(columns=['date','sqft_basement'],inplace=True)
predict_tab.drop(columns=['date','sqft_basement'],inplace=True)

In [ ]:
print("tabular",tabular.shape)
print("predict tabular",predict_tab.shape)

print("train_img",train_img.shape)
print("predict_img",predict_img.shape)

tabular (16110, 19)
predict tabular (5396, 18)
train_img (16110, 2049)
predict_img (5396, 2049)


In [ ]:
tabular.columns

Index(['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')

In [ ]:
predict_tab.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')

In [ ]:
#tabular columns
X_tab = tabular.drop(columns=['id','price'])
y_tab = tabular["price"]
X_tab_pred =predict_tab.drop(columns=['id'])

#image columns
X_img=train_img.drop(columns=['id'])
X_predict_img=predict_img.drop(columns=['id'])

In [ ]:
X_tab_train, X_tab_test, y_tab_train, y_tab_test = train_test_split(X_tab, y_tab,test_size=0.2,random_state=42)

# Transformers


In [ ]:
num_cols =['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']

In [ ]:
preprocessor = ColumnTransformer(transformers=[("num", StandardScaler(), num_cols)],remainder='passthrough')

In [ ]:
pipelines = {
    "Linear Regression": Pipeline([
        ("scaler", preprocessor),
        ("model", LinearRegression())
    ]),

    "Random Forest": Pipeline([
        ("scaler", preprocessor),
        ("model", RandomForestRegressor(
            n_estimators=300,
            random_state=42,
            n_jobs=-1
        ))
    ]),

    "XGBoost": Pipeline([
        ("scaler", preprocessor),
        ("model", XGBRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ))
    ]),

    "LightGBM": Pipeline([
        ("scaler", preprocessor),
        ("model", LGBMRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=8,
            num_leaves=31,
            random_state=42,
            n_jobs=-1
        ))
    ]),

    "HistGradientBoosting": Pipeline([
        ("scaler", preprocessor),
        ("model", HistGradientBoostingRegressor(
            max_depth=8,
            learning_rate=0.05,
            random_state=42
        ))
    ])
}


# Baseline model


In [ ]:
results = []

for name, pipe in pipelines.items():
    pipe.fit(X_tab_train, y_tab_train)

    y_train_pred = pipe.predict(X_tab_train)
    y_test_pred  = pipe.predict(X_tab_test)

    results.append({
        "Model": name,
        "Train R²": r2_score(y_tab_train, y_train_pred),
        "Test R²": r2_score(y_tab_test, y_test_pred),
        "MSE": mean_squared_error(y_tab_test, y_test_pred),
        "RMSE": np.sqrt(mean_squared_error(y_tab_test, y_test_pred))
    })


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2098
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 17
[LightGBM] [Info] Start training from score 538081.630897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
results_df = pd.DataFrame(results).sort_values("Test R²", ascending=False)
results_df


,Model,Train R²,Test R²,MSE,RMSE
2,XGBoost,0.974396,0.909377,1.107794e+10,105251.804127
3,LightGBM,0.948067,0.899738,1.225619e+10,110707.662022
4,HistGradientBoosting,0.906697,0.889618,1.349321e+10,116160.275893
1,Random Forest,0.983610,0.886334,1.389466e+10,117875.608559
0,Linear Regression,0.696012,0.704371,3.613817e+10,190100.429108


Best Model Pipeline

In [ ]:
best_pipeline = Pipeline([
    ("scaler", preprocessor),
    ("model", LGBMRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=8,
        num_leaves=31,
        random_state=42,
        n_jobs=-1
    ))
])



In [ ]:
best_pipeline.fit(X_tab_train, y_tab_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002213 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2098
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 17
[LightGBM] [Info] Start training from score 538081.630897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

Pipeline(steps=[('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'waterfront',
                                                   'view', 'condition', 'grade',
                                                   'sqft_above', 'yr_built',
                                                   'yr_renovated', 'zipcode',
                                                   'lat', 'long',
                                                   'sqft_living15',
                                                   'sqft_lot15'])])),
                ('model',
                 LGBMRegressor(learning_rate=0.05, max_depth=8,
                               n_estimators=400, n_jobs=-1, random_state=42))])

In [ ]:
y_test_pred = best_pipeline.predict(X_tab_test)

print("Final Test R² :", r2_score(y_tab_test, y_test_pred))
print("Final RMSE   :", np.sqrt(mean_squared_error(y_tab_test, y_test_pred)))


Final Test R² : 0.8997379787749298
Final RMSE   : 110707.66202157641


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


# Merge and PCA

In [ ]:
# Merge on id train.csv
df_mm = tabular.merge(train_img, on="id", how="inner")
print("Multimodal train:", df_mm.shape)

## Merge on id test.csv
predict_df_mm = predict_tab.merge(predict_img, on="id", how="inner")
print("Multimodal test:", predict_df_mm.shape)

Multimodal train: (16110, 2067)
Multimodal test: (5396, 2066)


In [ ]:
# Separate target
y = df_mm["price"]
X = df_mm.drop(columns=["id", "price"])

In [ ]:
tabular_cols = [
    'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
    'waterfront', 'view', 'condition', 'grade', 'sqft_above',
    'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
    'sqft_living15', 'sqft_lot15'
]

image_cols = [c for c in X.columns if c not in tabular_cols]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Scale image embeddings before PCA
img_scaler = StandardScaler()

X_train_img_scaled = img_scaler.fit_transform(X_train[image_cols])
X_test_img_scaled  = img_scaler.transform(X_test[image_cols])
X_predict_img_scaled = img_scaler.transform(predict_df_mm[image_cols])

# PCA
pca = PCA(n_components=80, random_state=42)

X_train_img_pca = pca.fit_transform(X_train_img_scaled) #train.csv
X_test_img_pca  = pca.transform(X_test_img_scaled) #train.csv

X_predict_img_pca =pca.transform(X_predict_img_scaled) #test.csv

print(f"Image PCA variance retained: {pca.explained_variance_ratio_.sum()*100:.2f}%")


Image PCA variance retained: 76.32%


In [ ]:
pca_cols = [f"img_pca_{i}" for i in range(X_train_img_pca.shape[1])]

X_train_mm = pd.concat(
    [
        X_train[tabular_cols].reset_index(drop=True),
        pd.DataFrame(X_train_img_pca, columns=pca_cols)
    ],
    axis=1
)

X_test_mm = pd.concat(
    [
        X_test[tabular_cols].reset_index(drop=True),
        pd.DataFrame(X_test_img_pca, columns=pca_cols)
    ],
    axis=1
)

X_predict_mm = pd.concat(
    [
        predict_df_mm[tabular_cols].reset_index(drop=True),
        pd.DataFrame(X_predict_img_pca, columns=pca_cols)
    ],
    axis=1
)

In [ ]:
print('train',X_train_mm.shape)
print('test',X_test_mm.shape)
print('predict',X_predict_mm.shape)

train (12888, 97)
test (3222, 97)
predict (5396, 97)


# Multimodal

In [ ]:
preprocessor = ColumnTransformer(transformers=[("tab", StandardScaler(), tabular_cols)],remainder='passthrough')

In [ ]:
pipelines = {
    "Linear Regression": Pipeline([
        ("prep", preprocessor),
        ("model", LinearRegression())
    ]),

    "Random Forest": Pipeline([
        ("prep", preprocessor),
        ("model", RandomForestRegressor(
            n_estimators=300, random_state=42, n_jobs=-1
        ))
    ]),

    "XGBoost": Pipeline([
        ("prep", preprocessor),
        ("model", XGBRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ))
    ]),

    "LightGBM": Pipeline([
        ("prep", preprocessor),
        ("model", LGBMRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=8,
            num_leaves=31,
            random_state=42
        ))
    ]),

    "HistGradientBoosting": Pipeline([
        ("prep", preprocessor),
        ("model", HistGradientBoostingRegressor(
            max_depth=8,
            learning_rate=0.05,
            random_state=42
        ))
    ])
}


In [ ]:
results = []

for name, pipe in pipelines.items():
    pipe.fit(X_train_mm, y_train)

    y_train_pred = pipe.predict(X_train_mm)
    y_test_pred  = pipe.predict(X_test_mm)

    results.append({
        "Model": name,
        "Train R²": r2_score(y_train, y_train_pred),
        "Test R²": r2_score(y_test, y_test_pred),
        "MSE": mean_squared_error(y_test, y_test_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred))
    })


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016290 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22498
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 97
[LightGBM] [Info] Start training from score 538081.630897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
results_df = pd.DataFrame(results).sort_values("Test R²", ascending=False)
results_df


,Model,Train R²,Test R²,MSE,RMSE
2,XGBoost,0.984170,0.905109,1.159960e+10,107701.427883
3,LightGBM,0.967950,0.898637,1.239082e+10,111314.049632
4,HistGradientBoosting,0.913704,0.883670,1.422036e+10,119249.165790
1,Random Forest,0.980475,0.866049,1.637433e+10,127962.222651
0,Linear Regression,0.719395,0.726388,3.344682e+10,182884.709582


# Predictions


Best Pipeline for multimodel

In [ ]:
best_pipeline = Pipeline([
    ("scaler", preprocessor),
    ("model", LGBMRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=8,
        num_leaves=31,
        random_state=42,
        n_jobs=-1
    ))
])


In [ ]:
best_pipeline.fit(X_train_mm, y_train)
predictions=best_pipeline.predict(X_predict_mm)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056868 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22498
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 97
[LightGBM] [Info] Start training from score 538081.630897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
predictions_df = pd.DataFrame({
    "id": predict_tab["id"].astype("int64"),
    "predicted_price": predictions
})


In [ ]:
predictions_df.to_csv("final_price_predictions.csv", index=False)